# Build a simulation using the Python API

Currently, this notbook manipulates the simulation by directly placing objects inside of the attributes of the network and domain. It should be refactored when proper methods exist for adding these objects.


Import the Simulation class

In [1]:
from primaite.simulator.sim_container import Simulation


Create an empty simulation. By default this has a network with no nodes or links, and a domain controller with no accounts.

Let's use the simulation's `describe_state()` method to verify that it is empty.

In [2]:
my_sim = Simulation()
net = my_sim.network
my_sim.describe_state()

{'uuid': '2ef348c6-32e5-4c5c-83b7-3b82d0b6123b',
 'network': {'uuid': 'dd2d1a02-d461-4505-8bbd-fd0681750175',
  'nodes': {},
  'links': {}},
 'domain': {'uuid': 'ae0423ee-51fa-41e7-be80-c642b39707f6', 'accounts': {}}}

## Add nodes

In [3]:
from primaite.simulator.network.hardware.base import Node


In [4]:
my_pc = Node(hostname="primaite_pc",)
net.add_node(my_pc)
my_server = Node(hostname="google_server")
net.add_node(my_server)


## Connect the nodes

In [5]:
from primaite.simulator.network.hardware.base import NIC, Link, Switch


In [6]:
my_swtich = Switch(hostname="switch1", num_ports=12)
net.add_node(my_swtich)

pc_nic = NIC(ip_address="130.1.1.1", gateway="130.1.1.255", subnet_mask="255.255.255.0")
my_pc.connect_nic(pc_nic)


server_nic = NIC(ip_address="130.1.1.2", gateway="130.1.1.255", subnet_mask="255.255.255.0")
my_server.connect_nic(server_nic)


net.connect(pc_nic, my_swtich.switch_ports[1])
net.connect(server_nic, my_swtich.switch_ports[2])


2023-08-24 13:06:28,617: NIC cc:be:ec:43:a6:4c/130.1.1.1 connected to Link cc:be:ec:43:a6:4c/130.1.1.1<-->79:2b:4a:70:c3:50
2023-08-24 13:06:28,618: SwitchPort 79:2b:4a:70:c3:50 connected to Link cc:be:ec:43:a6:4c/130.1.1.1<-->79:2b:4a:70:c3:50
2023-08-24 13:06:28,619: NIC c2:1e:48:e1:a4:ad/130.1.1.2 connected to Link c2:1e:48:e1:a4:ad/130.1.1.2<-->1a:2d:12:38:80:2f
2023-08-24 13:06:28,620: SwitchPort 1a:2d:12:38:80:2f connected to Link c2:1e:48:e1:a4:ad/130.1.1.2<-->1a:2d:12:38:80:2f


## Add files and folders to nodes


In [7]:
from primaite.simulator.file_system.file_type import FileType
from primaite.simulator.file_system.file_system import File

In [8]:
my_pc_downloads_folder = my_pc.file_system.create_folder("downloads")
my_pc_downloads_folder.add_file(File(name="firefox_installer.zip",file_type=FileType.ZIP))

In [9]:
my_server_folder = my_server.file_system.create_folder("static")
my_server.file_system.create_file("favicon.ico", file_type=FileType.PNG)

File(uuid='7d56a563-ecc0-4011-8c97-240dd6c885c0', name='favicon.ico', size=40.0, file_type=<FileType.PNG: '11'>, action_manager=None)

## Add applications to nodes

In [10]:
from primaite.simulator.system.applications.application import Application, ApplicationOperatingState
from primaite.simulator.system.software import SoftwareHealthState, SoftwareCriticality
from primaite.simulator.network.transmission.transport_layer import Port

# no applications exist yet so we will create our own.
class MSPaint(Application):
    def describe_state(self):
        return super().describe_state()

In [11]:
mspaint = MSPaint(name = "mspaint", health_state_actual=SoftwareHealthState.GOOD, health_state_visible=SoftwareHealthState.GOOD, criticality=SoftwareCriticality.MEDIUM, ports={Port.HTTP}, operating_state=ApplicationOperatingState.RUNNING,execution_control_status='manual')

In [12]:
my_pc.applications[mspaint.uuid] = mspaint

## Create a domain account

In [13]:
from primaite.simulator.domain.account import Account, AccountType


In [14]:
acct = Account(username="admin", password="admin12", account_type=AccountType.USER)
my_sim.domain.accounts[acct.uuid] = acct

Verify that the state dictionary contains no non-serialisable objects.

In [15]:
my_sim.describe_state()

{'uuid': '2ef348c6-32e5-4c5c-83b7-3b82d0b6123b',
 'network': {'uuid': 'dd2d1a02-d461-4505-8bbd-fd0681750175',
  'nodes': {'2f03b32b-7290-4921-8670-faebe4a19d63': {'uuid': '2f03b32b-7290-4921-8670-faebe4a19d63',
    'hostname': 'primaite_pc',
    'operating_state': 0,
    'NICs': {'e07e2a7f-b09f-4bd8-8e92-cffbf1f2270b': {'uuid': 'e07e2a7f-b09f-4bd8-8e92-cffbf1f2270b',
      'ip_adress': '130.1.1.1',
      'subnet_mask': '255.255.255.0',
      'gateway': '130.1.1.255',
      'mac_address': 'cc:be:ec:43:a6:4c',
      'speed': 100,
      'mtu': 1500,
      'wake_on_lan': False,
      'dns_servers': [],
      'enabled': False}},
    'file_system': {'uuid': '0b7206af-3e0a-41b0-8115-ae9e0dbbcd81',
     'folders': {'c161bc7c-9abd-4666-9b49-2745fdb65ebe': {'uuid': 'c161bc7c-9abd-4666-9b49-2745fdb65ebe',
       'name': 'downloads',
       'size': 1000.0,
       'files': {'f807d777-d167-4f37-9f9b-ced634af6ed5': {'uuid': 'f807d777-d167-4f37-9f9b-ced634af6ed5',
         'name': 'firefox_installer.z

In [16]:
import json
json.dumps(my_sim.describe_state())

'{"uuid": "2ef348c6-32e5-4c5c-83b7-3b82d0b6123b", "network": {"uuid": "dd2d1a02-d461-4505-8bbd-fd0681750175", "nodes": {"2f03b32b-7290-4921-8670-faebe4a19d63": {"uuid": "2f03b32b-7290-4921-8670-faebe4a19d63", "hostname": "primaite_pc", "operating_state": 0, "NICs": {"e07e2a7f-b09f-4bd8-8e92-cffbf1f2270b": {"uuid": "e07e2a7f-b09f-4bd8-8e92-cffbf1f2270b", "ip_adress": "130.1.1.1", "subnet_mask": "255.255.255.0", "gateway": "130.1.1.255", "mac_address": "cc:be:ec:43:a6:4c", "speed": 100, "mtu": 1500, "wake_on_lan": false, "dns_servers": [], "enabled": false}}, "file_system": {"uuid": "0b7206af-3e0a-41b0-8115-ae9e0dbbcd81", "folders": {"c161bc7c-9abd-4666-9b49-2745fdb65ebe": {"uuid": "c161bc7c-9abd-4666-9b49-2745fdb65ebe", "name": "downloads", "size": 1000.0, "files": {"f807d777-d167-4f37-9f9b-ced634af6ed5": {"uuid": "f807d777-d167-4f37-9f9b-ced634af6ed5", "name": "firefox_installer.zip", "size": 1000.0, "file_type": "ZIP"}}, "is_quarantined": false}}}, "applications": {"ea466b2f-1ed5-49fd